In [1]:
!pip install pandas numpy matplotlib seaborn
!pip install torch torchvision torchaudio
!pip install transformers
!pip install streamlit

In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
!pip install -U datasets
# Load IMDb dataset
from datasets import load_dataset
dataset = load_dataset("imdb")

# Combine train and test sets
df = pd.concat([pd.DataFrame(dataset["train"]), pd.DataFrame(dataset["test"])])

#Reduce Rows
df = df.sample(5000, random_state=42)

# Add Neutral examples (label = 2) for demonstration
neutral_df=df.sample(1000,random_state=42).copy()
neutral_df['label']=2

# Final dataset: Positive = 1, Negative = 0, Neutral = 2
df_final = pd.concat([df,neutral_df]).reset_index(drop=True)
df_final = df_final.sample(frac=1).reset_index(drop=True)

# Preview
df_final.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,text,label
0,STAR RATING: ***** The Works **** Just Misses ...,2
1,This is a beautiful film. The true tale of bon...,1
2,"It is nice to see the likes of Oliver Stone, B...",0
3,This is one of my favorite T.V shows of all ti...,1
4,This film was recommended to me by a friend wh...,0


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(texts,labels,max_length=128):
    encodings = tokenizer(texts,truncation=True,padding=True,max_length=max_length)
    input_ids = torch.tensor(encodings['input_ids'])
    attention_mask = torch.tensor(encodings['attention_mask'])
    labels = torch.tensor(labels)
    return input_ids,attention_mask,labels

x_train,x_val,y_train,y_val=train_test_split(
    df_final['text'].tolist(),df_final['label'].tolist(),test_size=0.1,random_state=42)

train_inputs,train_masks,train_labels=tokenize_data(x_train,y_train)
val_inputs,val_masks,val_labels=tokenize_data(x_val,y_val)

In [5]:
class SentimentDataset(Dataset):
    def __init__(self,input_ids,attention_masks,labels):
        self.input_ids=input_ids
        self.attention_masks=attention_masks
        self.labels=labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self,idx):
        return {
            'input_ids':self.input_ids[idx],
            'attention_mask':self.attention_masks[idx],
            'labels':self.labels[idx]
        }

train_data= SentimentDataset(train_inputs,train_masks,train_labels)
val_data= SentimentDataset(val_inputs,val_masks,val_labels)

train_loader= DataLoader(train_data,batch_size=8,shuffle=True)
val_loader= DataLoader(val_data,batch_size=8)

In [6]:
device = torch.device('cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)
model.to(device)

optimizer = AdamW(model.parameters(),lr=2e-5)

#Training Loop
for epoch in range(3):
    model.train()
    for batch in train_loader:
        b_input_ids = batch['input_ids'].to(device)
        b_attention_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(b_input_ids, attention_mask = b_attention_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} done.")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 done.
Epoch 2 done.
Epoch 3 done.


In [11]:
model.eval()
preds, true = [], []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids,attention_mask=attention_mask)
        logits = outputs.logits
        preds += torch.argmax(logits, axis=1).cpu().tolist()
        true += labels.cpu().tolist()

print(classification_report(true, preds, target_names=['Negative', 'Positive', 'Neutral']))

              precision    recall  f1-score   support

    Negative       0.77      0.85      0.81       270
    Positive       0.71      0.89      0.79       241
     Neutral       0.00      0.00      0.00        89

    accuracy                           0.74       600
   macro avg       0.49      0.58      0.53       600
weighted avg       0.63      0.74      0.68       600



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
#Save the Model
model.save_pretrained('sentiment_model')
tokenizer.save_pretrained('sentiment_maodel')

('sentiment_maodel/tokenizer_config.json',
 'sentiment_maodel/special_tokens_map.json',
 'sentiment_maodel/vocab.txt',
 'sentiment_maodel/added_tokens.json')